# Incident example

In this example, we will use a considerably complex graph that simulates a microservice architecture to perform Weak PN and Weak PS queries

In [ ]:
import logging
import os

import networkx as nx
import pandas as pd

from pici.causal_model import CausalModel
from pici.utils._enum import DataExamplesPaths

THIS_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(THIS_DIR, ".."))
import sys
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

## Defining the model

In [ ]:
edges_list_2 = [
    ("DB_Change", "DB_Latency"),
    ("DB_Latency", "MS-B_Latency"),
    ("MS-B_Latency", "MS-B_Error"),
    ("MS-B_Latency", "MS-A_Latency"),
    ("MS-B_Error", "MS-A_Error"),
    ("MS-A_Latency", "MS-A_Threads"),
    ("MS-A_Threads", "MS-A_Crash"),
    ("MS-A_Error", "Outage"),
    ("MS-A_Crash", "Outage"),
    ("HeavyTraffic", "MS-B_Latency"),
    ("HeavyTraffic", "MS-A_Latency"),
    # UNOBS
    ("Unob_helper_1", "DB_Change"),
    ("Unob_helper_2", "DB_Latency"),
    ("Unob_helper_3", "MS-B_Error"),
    ("Unob_helper_4", "MS-A_Error"),
    ("Unob_helper_5", "MS-A_Threads"),
    ("Unob_helper_6", "MS-A_Crash"),
    ("Unob_helper_7", "Outage"),
]

latent_nodes_2 = [
    "HeavyTraffic",
    "Unob_helper_1",
    "Unob_helper_2",
    "Unob_helper_3",
    "Unob_helper_4",
    "Unob_helper_5",
    "Unob_helper_6",
    "Unob_helper_7",
]
edges_2 = nx.DiGraph(edges_list_2)
df_medium_scale_incident = pd.read_csv(
    os.path.join(PROJECT_ROOT, DataExamplesPaths.NEW_MEDIUM_SCALE_OUTAGE_INCIDENT.value)
)
model_2 = CausalModel(
    data=df_medium_scale_incident,
    edges=edges_2,
    unobservables_labels=latent_nodes_2,
)

## Visualizing the graph

In [ ]:
nx.draw_networkx(model_2.graph.DAG)

## Generating an image to save

In [ ]:
model_2.generate_graph_image("model_2.png")

## Calculating the Weak PN and Weak PS queries

In [ ]:
intervention_1 = "MS-B_Latency"
intervention_2 = "DB_Latency"
target = "Outage"

print(
    f"Intervention: {intervention_1} Target: {target} --> Weak-PN = {model_2.weak_pn_inference(intervention_label=intervention_1, target_label=target)}"
)

print(
    f"Intervention: {intervention_1} Target: {target} --> Weak-PS = {model_2.weak_ps_inference(intervention_label=intervention_1, target_label=target)}"
)

print(
    f"Intervention: {intervention_2} Target: {target} --> Weak-PN = {model_2.weak_pn_inference(intervention_label=intervention_2, target_label=target)}"
)

print(
    f"Intervention: {intervention_2} Target: {target} --> Weak-PS = {model_2.weak_ps_inference(intervention_label=intervention_2, target_label=target)}"
)